# Customer value modeling

You are given a CSV file representing approximately 50,000 transactions from an ecommerce store with the goal to predict future purchasing behavior. Each row represents a purchase. The first column is the `CustomerID` and it’s possible to have multiple rows with the same `CustomerID`. The second column is a UTC timestamp of the transaction and the third column is the monetary value of the transaction.

To do this exercise, use the modified beta geometric and the gamma gamma model in the lifetimes python library (https://github.com/CamDavidsonPilon/lifetimes). Consider the work of Fader and Hardie (cited in the python package) while implementing your model. Keep in mind there are very advanced models that could take weeks or months to tune, but for this exercise, stick to what has already been implemented in the lifetimes library.

### Questions

1. Implement the modified BG model from the lifetimes package using the data we provide.
2. List the 10 customers predicted to make the most purchases over the next 12 months.
3. List the 10 customers predicted to spend the most over the next 12 months.
4. Explain the statistical assumptions that the model makes. Do you think those assumptions are valid for a model of a customer buying widgets from an ecommerce store?
5. Write a simulation that shows how many customers are alive after 10 days, 1 year, 10 years and 100 years and how many purchases they have made in that time using the modified BG model. Use a simple random sample of 100 customers and show the results for 1 run of your simulation. We’ve outlined a possible approach below:

> `a. Generate a random sample of 100 customers. <br>
> b. Simulate how each customer makes purchases over time. <br>
> c. Count how many purchases the customers have made in 10 days. <br>
> d. Count how many customers are alive after 10 days. <br>
> e. Repeat b-d for 1 year, 10 years, 100 years. <br>
> f. Tip: Your simulation should show a different number of customers alive each time the code is evaluated.  (Often it is interesting to look at this range of values, but for this exercise, we are only looking for you to show us the result of one run.) <br>
> g. Tip: You will have to write code beyond what is available in the python package.` <br>

6. The model works well in some ways and poorly in others for the data we provide. Explain what it does a well and what it does poorly.
7. What are one or two ideas that could address the deficiencies of the model? (No need to implement, just describe.)

In [4]:
import pandas as pd
import numpy as np
from lifetimes import ModifiedBetaGeoFitter, GammaGammaFitter
from lifetimes.utils import summary_data_from_transaction_data

**1. Implement the modified BG model from the lifetimes package.**

In [5]:
url = 'https://raw.githubusercontent.com/edvinmemet/projects/master/data/customer_modeling/customer_modeling.csv?token=AF52KYIP2TEYYLNAPIBEIGS5VJQAK'
TransactionData = pd.read_csv(url) 
TransactionData['Timestamp'] = TransactionData['Timestamp'].str[:-2] #remove timezone placeholder
SummaryData = summary_data_from_transaction_data(TransactionData, 'CustomerID', 'Timestamp', 'PurchaseValue') 
mbgf = ModifiedBetaGeoFitter(penalizer_coef = 0)
mbgf.fit(SummaryData['frequency'], SummaryData['recency'], SummaryData['T']);

**2. List the 10 customers predicted to make the most purchases over the next 12 months.**

In [6]:
days_in_year = 365
SummaryData['predicted_purchases'] = mbgf.predict(days_in_year, SummaryData['frequency'], SummaryData['recency'], SummaryData['T'])
SummaryData.sort_values(by='predicted_purchases', ascending = False).head(10)

,frequency,recency,T,monetary_value,predicted_purchases
CustomerID,,,,,
a5fadc51b1ae844ad9a70ad3cfa46a64,95.0,371.0,371.0,433.962632,64.472898
48a503edbaded96a3be27deee11967a1,46.0,363.0,371.0,220.476739,30.920429
9f447f9415a380ac2eeee7df49c6ee7e,23.0,163.0,166.0,110.196522,25.950894
5f01420f0edda6555df5ce1cc62b986c,39.0,350.0,370.0,299.122821,25.056529
8d2ce54737dd404d20cadf1405d46dc8,33.0,360.0,362.0,123.195152,22.918299
a62a17bb46864da2c6da691d838971b3,30.0,366.0,366.0,307.961000,20.753380
3b11478939967e896ae2619615650f97,20.0,190.0,194.0,58.866000,20.734257
2ad9a83ee23110d8c2f4c01600b94f20,29.0,357.0,364.0,146.420690,19.942978
75fda9ea22086bf3814ff8c3f53de8ca,28.0,348.0,355.0,196.032857,19.602512


**3. List the 10 customers predicted to spend the most over the next 12 months.**

In [7]:
ggf = GammaGammaFitter(penalizer_coef = 0) #initialize gamma gamma model and fit on returning customers
ggf.fit(SummaryData.loc[SummaryData['frequency']>0, 'frequency'], SummaryData.loc[SummaryData['frequency']>0, 'monetary_value'])

SummaryData['predicted_average_monetary_value']= ggf.conditional_expected_average_profit(SummaryData['frequency'], SummaryData['monetary_value'])
#multiply predicted average purchase value (from GG model) by the predicted number of purchases (from MBG model)
SummaryData['predicted_spending'] = SummaryData['predicted_purchases'] * SummaryData['predicted_average_monetary_value']
SummaryData.sort_values(by='predicted_spending', ascending = False).head(10)

,frequency,recency,T,monetary_value,predicted_purchases,predicted_average_monetary_value,predicted_spending
CustomerID,,,,,,,
a5fadc51b1ae844ad9a70ad3cfa46a64,95.0,371.0,371.0,433.962632,64.472898,432.751074,27900.715844
ca2202a96c2de6ca6b8a37a4a73fa730,7.0,12.0,20.0,1399.634286,15.618740,1310.471093,20467.906824
dca76db00cc59dfbcdcc97c8bbc7f9f1,19.0,339.0,364.0,966.423158,12.625242,944.737335,11927.537437
5ac5ed64cd99ed2a8403b7a927e644ef,23.0,222.0,250.0,627.439565,17.918448,617.769916,11069.478204
60c19a709e3ced2d16d7100eb1069df5,10.0,292.0,300.0,1259.266000,8.072716,1203.080039,9712.123519
98f8e41f45721cbe49a3147f6cf62432,3.0,39.0,71.0,1757.860000,5.013617,1511.038684,7575.769641
5f01420f0edda6555df5ce1cc62b986c,39.0,350.0,370.0,299.122821,25.056529,298.172878,7471.177408
a92534133444b5028d12a129b1b128f7,6.0,66.0,75.0,772.826667,10.049583,725.359985,7289.565500
089ecc49200cfe79584d0bec2a3cf8c0,2.0,125.0,134.0,2937.115000,3.112713,2329.534697,7251.174079


**4. Explain the statistical assumptions that the model makes. Do you think those assumptions are valid for a model of a customer buying widgets from an ecommerce store?**

Statistical assumptions of the MBG model:


* Immediately following every transaction (including after the initial/'trial' transaction, which establishes the time origin t=0 for each customer), customers can "die" with probability $p$, where $p$ is a random variable which has a Beta distribution across customers $p \sim \mathrm{Beta}(a,b)$.

 
* For active customers, transactions follow a Poisson process with rate $\lambda$ drawn from a Gamma distribution with shape $r$ and inverse scale $\alpha$: $ \lambda \sim \Gamma(r, 1/\alpha)$. 


* Transaction rate $\lambda$ and dropout probability $p$ vary independently accross customers. 


Statistical assumptions of the GG model:

* Each customer is characterised by an unobserved mean transaction value $\zeta$, which does not vary over time for a given individual but does vary across customers


* For a given customer, the monetary value of their transactions $Z_i$ is drawn from a Gamma distribution: $Z_i \sim \Gamma ($p$, \nu)$, with the mean transaction value converging to the underlying mean: $E \left[\bar z\right] = \zeta$. Note that $p$ (not to be confused with the $p$ of the MBG model!) is fixed accross customers (but unknown), while $\nu$ is a random variable drawn from another Gamma distribution: $\nu \sim \Gamma (q, \gamma)$


* Transaction values are independent of frequency and recency. Thus, for example, frequent buyers do not tend to spend more/less per transaction than infrequent ones. 


Comments on the validity of these assumptions:

* One of the main assumptions is that of independence between the monetary value of a transaction and frequency/recency, which can be checked by examining whether the correlation coefficients are close to zero. For example, for our data, the correlation coefficients between monetary value and frequency/recency are both very low (under 0.1) when we consider only returning customers (since customers without repeat transactions have a monetary value of zero by default). While the validity of this assumption needs to be checked explicitly in each case, it does appear that the assumption is approximately valid in many typical cases.

* Another important assumption of the model is that the dropout probability $p$ is independent of the transaction rate $\lambda$. However, it's not clear how realistic this assumption is: intuitively we might expect a frequent customer to be less likely to "die" than an infrequent one. This issue with the model is compounded by the fact that dropout can only occur immediately after transactions; thus, in this model, given a frequent customer and an infrequent one with the same dropout probability $p$, the frequent one is actually more likely to die in a given interval due to his higher transaction volume.

* Similarly, we might expect intuitively that dropout probability $p$ will decrease with increasing customer age (i.e. more loyal customers are less likely to dropout). Such an effect cannot be captured in the MBG model, where dropout probability $p$ is fixed over a customer's lifetime. 


* For customers buying widgets from an ecommerce store, the assumptions of the MBG and GG models should be quite reasonable for most type of "widgets" (if the independence of monetary value and frequency/recency is confirmed). Note that for widgets that have a long lifetime (so that they don't need to be purchased frequently), the model will be most useful if the observation/calibration period is long enough compared to the mean lifetime/interpurchase time of the widget. In particular, in the case of widgets that customers only need to buy, say, about once in their lifetime (e.g. a tool box), the model would not prove very useful at all. 

* In addition, for the models to work well for the widget store, the number of customers should be sufficiently large. If the customer base of the ecommerce store is small, or their transaction volume is low, the models may not perform as well. 

**5. Write a simulation that shows how many customers are alive after 10 days, 1 year, 10 years and 100 years and how many purchases they have made in that time using the modified BG model. Use a simple random sample of 100 customers and show the results for 1 run of your simulation.**

In [5]:
deltaT = [10, 1*days_in_year, 10*days_in_year, 100*days_in_year]
n_sample, n_alive, n_purchases = 100, [], []

SummaryDataSample = SummaryData[['frequency', 'recency', 'T']].sample(n=n_sample).reset_index() #sample customers
r_updated = mbgf.params_.r + SummaryDataSample['frequency']
alpha_updated = mbgf.params_.alpha + SummaryDataSample['T']
a_updated = mbgf.params_.a
b_updated = mbgf.params_.b + SummaryDataSample['frequency']
#sample lambda and p for each customer (conditional on their RFM) using updated parameters r, alpha, a, b
SummaryDataSample['lambda'] = np.random.gamma(r_updated, 1/alpha_updated, size = n_sample)
SummaryDataSample['p'] = np.random.beta(a_updated, b_updated, size = n_sample)
    
for dT in deltaT:
    SummaryDataSample['frequency'] = 0 #use this column to count purchases
    SummaryDataSample['alive'] = True #use this column to count alive customers
    for i in range(n_sample):
        #get the parameters for the given customer
        p, lambda_ = SummaryDataSample.at[i, 'p'], SummaryDataSample.at[i, 'lambda']
        tx, T = SummaryDataSample.at[i, 'recency'], SummaryDataSample.at[i, 'T']
        #compute initial probability of being alive
        p_alive_initial = (1-p) * np.exp(-lambda_ *(T-tx)) / (p + (1-p) * np.exp(-lambda_ *(T-tx))) # probability of having died after the last transaction
        if np.random.uniform() > p_alive_initial:
            SummaryDataSample.at[i, 'alive'] = False
        else: #customer is still alive at the start of the simulation
            time_last_active = T
            while SummaryDataSample.at[i, 'alive'] and time_last_active < T + dT: #while customer is alive and within time window dT
                time_to_next_purchase = np.random.exponential(1/lambda_)
                time_last_active += time_to_next_purchase
                if time_last_active < T + dT: #if the new transaction falls within the time interval dT
                    SummaryDataSample.at[i, 'frequency'] += 1 #update the number of purchases
                    if np.random.uniform() < p:
                        SummaryDataSample.at[i, 'alive'] = False #customer has died
    n_alive.append(sum(SummaryDataSample['alive'])) #store the number of alive customers for this value of dT
    n_purchases.append(sum(SummaryDataSample['frequency'])) #store the number of purchases for this value of dT

Results = pd.DataFrame([n_alive, n_purchases], columns = ['10 days', '1 year', '10 years', '100 years'], index = ['Customers alive', 'Number of purchases'])
Results #show the dataframe with the simulation results 

,10 days,1 year,10 years,100 years
Customers alive,74,51,31,11
Number of purchases,3,145,616,1495


### Comments on the simulation:


For each customer in the sample, their last transaction took place at time $t_x$. Following this transaction, the customer will have either died with probability $p$, or otherwise survived and made no other transactions during the time interval $(t_x, \, T]$. Given that our simulation starts at time $T$, we have to "flip a coin" for each customer to determine whether they are still alive or have died. However, the probability of having died is not simply $p$, since we also must incorporate the information that the customer made no transaction in the interval $(t_x, \, T]$. For example, a customer with $t_x = 20$ and $T= 300$ is more likely to have died than a customer with $t_x = 230$ and $T=300$. Since the probability of the customer being alive and having zero transactions in the interval $(t_x, \, T]$ is $(1-p) \, e^{-\lambda (T-t_x)}$, the initial probability of the customer being alive can be written as:

$$ \displaystyle p^{\mathrm{alive}}_{\mathrm{initial}} = \frac{\left(1-p\right) e^{-\lambda (T-t_x)}}{ p + \left(1-p\right) e^{-\lambda (T-t_x)}}$$

Once we flip this initial coin, we determine whether the customer is alive or not and in case he is, we run the simulation until either the customer dies or exceeds the simulation time window. Since transactions follow a Poisson process with rate $\lambda$, the time between transactions is exponentially distributed with rate $\lambda$. Therefore, we draw from this exponential distribution to determine the time interval between purchases and then flip a coin to determine if the customer dies (with probability $p$). 

Note that each customer is assigned values for $\lambda$ and $p$, which are sampled from a Gamma and a Beta distribution, respectively. The parameters of these distributions, $r$, $\alpha$, $a$, and $b$ were determined by fitting the MBG model on the entire data set. (Another possibility would be to run the fit again on the sample of 100 customers, in order to get parameters that better represent the sample; however, this doesn't seem like a good idea given the small sample size). When drawing $\lambda$ and $p$ for each customer, we have to be careful, again, about the prior information (RFM): $(X=x, t_x, T)$. Intuitively, a customer with a high purchase frequency is more likely to have a higher value of $\lambda$ than a customer with, say, only two repeat purchases over $T=100$ days. Fortunately, taking the RFM information into account is equivalent to simply updating the parameters of the distributions for $\lambda$ and $p$ as follows: $r' = r+x, \,\,\, \alpha' =  \alpha + T, \,\,\, a' = a, \,\,\, b' = b + x$ (see _Wagner, U. and Hoppe, D., 2008. Erratum on the MBG/NBD Model_ as well as Appendix A in _Batislam, E.P., Denizel, M. and Filiztekin, A., 2007. Empirical validation and comparison of models for customer base analysis._). Therefore, we sample $\lambda$ and $p$ for each customer from their respective distributions with updated parameters: $\lambda \sim \Gamma(r', 1/\alpha'), \,\,\, p \sim \mathrm{Beta}(a',b')$. 

** [Should we wish, instead, to start the simulation at $t=0$, effectively throwing away the RFM information for each customer, we can simply do that in the code by removing the parameter update terms in lines 5-8 and setting $p_{\mathrm{alive}}^{\mathrm{initial}} = 1-p$. However, my concern with starting the simulation at $t=0$ is that we would be throwing away _all_ customer information $(X=x, t_x, T)$, effectively rendering the sampling step useless (i.e. it makes no difference _which_ 100 customers we sample if we're throwing away all of their data and sampling $\lambda$, $p$ from their unconditional priors)]

**6. The model works well in some ways and poorly in others for the data we provide. Explain what it does well and what it does poorly.**

The model is powerful in its simplicity: simple behavioral rules and assumptions provide a good fit to data as well as good predictions _at the aggregate level_. Moreover, the simplicity of the model allows for an intuitive understanding of its predictions and implications as well as for a reduction in data storage requirements (since we only need RFM for each customer rather than their entire transaction history). Overall, given a sufficiently long training period, the model is very good at capturing _aggregate behavior_ including, for example, predicting the average number of transactions in the holdout period for customers with a given number of transactions in the calibration period (e.g. using ``` plot_calibration_purchases_vs_holdout_purchases ```). 

<br>

However, the variability in, say, holdout purchases accross customers with the same number of calibration purchases is still substantial, making it difficult to make accurate predictions at level of individual customers. 

Another issue is that, over longer periods of time, customers may adjust their behavior; however, the model cannot capture, for example, changes in the purchase rate, dropout probability, or monetary value over time (e.g. more loyal customers buy more) or cyclical changes in these parameters (e.g. weekly or yearly patterns of purchase). Thus, extrapolating the model over too long a time period may not be very realistic. In addition, the model is passive in that it does not consider customer incentivization/promotions and the response of customers to these actions. 

Regarding the Gamma-Gamma model, it may be rather poor in making predictions for what is typically the largest class of customers, those with zero repeat purchases. Namely, Gamma-Gamma simply predicts the expected monetary value per transaction for these customers to be the mean of all transaction values. A better prediction, perhaps, would also incorporate the monetary value of the trial purchase, which is currently ignored.


**7. What are one or two ideas that could address the deficiencies of the model? (No need to implement, just describe.)**

* Examine ways of relaxing some of the constraints associated with the dropout probability parameter $p$, such as allowing for coupling between $p$ and transaction rate $\lambda$ (e.g. customers who purchase very frequently are less likely to drop out) or between $p$ and customer age or total number of transactions (e.g. with time, customers become more loyal and thus less likely to drop out). In particular, as mentioned before, it seems that a weakness of the model is that, given two customers with equal dropout probabilities $p$, the one with higher transaction rate is in fact more likely to die within a given time interval, since they will make more purchases and, thus, have more opportunities to die. 


* Relax the assumption of independence between the distribution of monetary value and the underlying transaction process to allow for a correlation between $\nu$ and $\lambda$ (e.g. by replacing their respective distributions with a bivariate distribution with gamma marginals; see _Fader, P.S., Hardie, B.G. and Lee, K.L., 2005. RFM and CLV: Using iso-value curves for customer base analysis._)


